In [1]:
print('Importing packages...')
from datetime import datetime
import numpy as np
import pandas as pd
import xgboost
import re
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

Importing packages...


In [2]:
print('Reading data...')
dfTest = pd.read_csv('../input/loan-defaulter-classification/test_indessa.csv')
dfTrain = pd.read_csv('../input/loan-defaulter-classification/train_indessa.csv')

dfTrain = dfTrain[['member_id', 'loan_amnt', 'funded_amnt', 'addr_state', 'funded_amnt_inv', 'sub_grade', 'term', 'desc', 'emp_length', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc','total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'last_week_pay', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'loan_status']]
dfTest = dfTest[['member_id', 'loan_amnt', 'funded_amnt', 'addr_state', 'funded_amnt_inv', 'sub_grade', 'term', 'desc','emp_length', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'last_week_pay', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim']]

Reading data...


### Data Transformation

In [3]:

def get_last_week_pay(raw) :
    try :
        return int(re.sub("\D", "", raw))
    except :
        return -9999
    
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
def clean_text(raw_text):
    cleantext = np.nan
    if type(raw_text) == str :
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, ' ', raw_text)
        cleantext = cleantext.replace('>', '')
        cleantext = ' '.join(cleantext.split())
        
        stop_words = set(stopwords.words("english"))
        words = word_tokenize(cleantext)
        
        filtered_sentence = []

        for w in words:
            if w not in stop_words:
                filtered_sentence.append(w)
        return len(filtered_sentence)
    
    else :
        return 0 


In [4]:
'''
Data transformation/cleanup
Strip off textual parts, represent values as numeric values
it makes sense. Convert the datatype to numeric.
'''

dfTrain['term'] = dfTrain['term'].apply(lambda x : int(re.sub("\D", "", x)))
dfTrain['last_week_pay'] = dfTrain['last_week_pay'].apply(get_last_week_pay)
dfTrain['desc'] = dfTrain['desc'].apply(clean_text)
dfTrain['emp_length'].replace('n/a', '0', inplace=True)
dfTrain['emp_length'].replace(to_replace='\+ years', value='', regex=True, inplace=True)
dfTrain['emp_length'].replace(to_replace=' years', value='', regex=True, inplace=True)
dfTrain['emp_length'].replace(to_replace='< 1 year', value='0', regex=True, inplace=True)
dfTrain['emp_length'].replace(to_replace=' year', value='', regex=True, inplace=True)
dfTest['term'] = dfTest['term'].apply(lambda x : int(re.sub("\D", "", x)))
dfTest['last_week_pay'] = dfTest['last_week_pay'].apply(get_last_week_pay)
dfTest['desc'] = dfTest['desc'].apply(clean_text)
dfTest['emp_length'].replace(to_replace='\+ years', value='', regex=True, inplace=True)
dfTest['emp_length'].replace(to_replace=' years', value='', regex=True, inplace=True)
dfTest['emp_length'].replace(to_replace='< 1 year', value='0', regex=True, inplace=True)
dfTest['emp_length'].replace(to_replace=' year', value='', regex=True, inplace=True)

dfTrain['term'] = pd.to_numeric(dfTrain['term'], errors='coerce')
dfTest['term'] = pd.to_numeric(dfTest['term'], errors='coerce')

dfTrain['last_week_pay'] = pd.to_numeric(dfTrain['last_week_pay'], errors='coerce')
dfTest['last_week_pay'] = pd.to_numeric(dfTest['last_week_pay'], errors='coerce')

dfTrain['emp_length'] = pd.to_numeric(dfTrain['emp_length'], errors='coerce')
dfTest['emp_length'] = pd.to_numeric(dfTest['emp_length'], errors='coerce')

dfTrain['sub_grade'] = pd.to_numeric(dfTrain['sub_grade'], errors='coerce')
dfTest['sub_grade'] = pd.to_numeric(dfTest['sub_grade'], errors='coerce')

print('Transform done.')

Transform done.


In [5]:
'''
Feature Engineering
'''

# Separating the member_id column of test dataframe to help create a csv after predictions
test_member_id = pd.DataFrame(dfTest['member_id'])


# Creating target variable pandas series from train dataframe, this will be used by cross validation to calculate
# the accuracy of the model
train_target = pd.DataFrame(dfTrain['loan_status'])


# It's good to create a copy of train and test dataframes. this way we can play around different features as we tune the
# performance of the classifier with important features
selected_cols = ['member_id', 'emp_length', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'sub_grade', 'int_rate', 'annual_inc', 'dti', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'revol_bal', 'revol_util', 'total_acc', 'total_rec_int', 'total_rec_late_fee', 'mths_since_last_major_derog', 'last_week_pay', 'tot_cur_bal', 'total_rev_hi_lim', 'tot_coll_amt', 'recoveries', 'collection_recovery_fee', 'term', 'acc_now_delinq', 'collections_12_mths_ex_med']
finalTrain = dfTrain[selected_cols]
finalTest = dfTest[selected_cols]

# How big the loan a person has taken with respect to his earnings, annual income to loan amount ratio
finalTrain['loan_to_income'] = finalTrain['annual_inc']/finalTrain['funded_amnt_inv']
finalTest['loan_to_income'] = finalTest['annual_inc']/finalTest['funded_amnt_inv']


# All these attributes indicate that the repayment was not all hunky-dory. All the amounts caclulated are ratios 
# like, recovery to the loan amount. This column gives a magnitude of how much the repayment has gone off course 
# in terms of ratios.
finalTrain['bad_state'] = finalTrain['acc_now_delinq'] + (finalTrain['total_rec_late_fee']/finalTrain['funded_amnt_inv']) + (finalTrain['recoveries']/finalTrain['funded_amnt_inv']) + (finalTrain['collection_recovery_fee']/finalTrain['funded_amnt_inv']) + (finalTrain['collections_12_mths_ex_med']/finalTrain['funded_amnt_inv'])
finalTest['bad_state'] = finalTest['acc_now_delinq'] + (finalTest['total_rec_late_fee']/finalTest['funded_amnt_inv']) + (finalTest['recoveries']/finalTest['funded_amnt_inv']) + (finalTest['collection_recovery_fee']/finalTest['funded_amnt_inv']) + (finalTrain['collections_12_mths_ex_med']/finalTest['funded_amnt_inv'])

# For the sake of this model, I have used just a boolean flag if things had gone bad, with this case I didn't see
# a benifit of including above computations
finalTrain.loc[finalTrain['bad_state'] > 0, 'bad_state'] = 1
finalTest.loc[finalTest['bad_state'] > 0, 'bad_state'] = 1


# Total number of available/unused 'credit lines'
finalTrain['avl_lines'] = finalTrain['total_acc'] - finalTrain['open_acc']
finalTest['avl_lines'] = finalTest['total_acc'] - finalTest['open_acc']


# Interest paid so far
finalTrain['int_paid'] = finalTrain['total_rec_int'] + finalTrain['total_rec_late_fee']
finalTest['int_paid'] = finalTest['total_rec_int'] + finalTest['total_rec_late_fee']


# Calculating EMIs paid (in terms of percent)
finalTrain['emi_paid_progress_perc'] = ((finalTrain['last_week_pay']/(finalTrain['term']/12*52+1))*100)
finalTest['emi_paid_progress_perc'] = ((finalTest['last_week_pay']/(finalTest['term']/12*52+1))*100)


# Calculating total repayments received so far, in terms of EMI or recoveries after charge off
finalTrain['total_repayment_progress'] = ((finalTrain['last_week_pay']/(finalTrain['term']/12*52+1))*100) + ((finalTrain['recoveries']/finalTrain['funded_amnt_inv']) * 100)
finalTest['total_repayment_progress'] = ((finalTest['last_week_pay']/(finalTest['term']/12*52+1))*100) + ((finalTest['recoveries']/finalTest['funded_amnt_inv']) * 100)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [6]:
#Null values in training dataset

null= finalTrain.isnull().sum().sort_values(ascending=False)
total =finalTrain.shape[0]
percent_missing= (finalTrain.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})
 
print ("Null Values in each column:\n", missing_data.sort_values(by ='Total missing', ascending = False))

Null Values in each column:
                     column name  Total missing  Percent missing
0                     sub_grade         532428         1.000000
1        mths_since_last_record         450305         0.845758
2   mths_since_last_major_derog         399448         0.750239
3        mths_since_last_delinq         272554         0.511908
4                  tot_coll_amt          42004         0.078891
5              total_rev_hi_lim          42004         0.078891
6                   tot_cur_bal          42004         0.078891
7                    emp_length          26891         0.050506
8                    revol_util            287         0.000539
9                     bad_state            228         0.000428
10     total_repayment_progress            113         0.000212
11   collections_12_mths_ex_med             95         0.000178
12                    total_acc             16         0.000030
13               acc_now_delinq             16         0.000030
14         

In [7]:
#Null values in training dataset

null= finalTest.isnull().sum().sort_values(ascending=False)
total =finalTest.shape[0]
percent_missing= (finalTest.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})
 
print ("Null Values in each column:\n", missing_data.sort_values(by ='Total missing', ascending = False))

Null Values in each column:
                     column name  Total missing  Percent missing
0                     sub_grade         354951         1.000000
1        mths_since_last_record         300021         0.845246
2   mths_since_last_major_derog         266228         0.750042
3        mths_since_last_delinq         181758         0.512065
4                  tot_coll_amt          28272         0.079650
5              total_rev_hi_lim          28272         0.079650
6                   tot_cur_bal          28272         0.079650
7                    emp_length          17934         0.050525
8                    revol_util            215         0.000606
9                     bad_state            170         0.000479
10     total_repayment_progress             84         0.000237
11   collections_12_mths_ex_med             50         0.000141
12                    total_acc             13         0.000037
13               acc_now_delinq             13         0.000037
14         

In [8]:
def fill_nulls(value):
    cols_fill = ['mths_since_last_record','mths_since_last_major_derog',
                 'mths_since_last_delinq','total_rev_hi_lim','tot_cur_bal',
                 'tot_coll_amt','emp_length','revol_util','collections_12_mths_ex_med',
                 'open_acc','total_acc','acc_now_delinq','avl_lines','loan_to_income',
                 'annual_inc','bad_state','total_repayment_progress']
    
    if value == -9999:
        for col in cols_fill:
            finalTest.loc[finalTest[col].isnull(), col] = -9999
    else : 
        for col in cols_fill:
            finalTest.loc[finalTest[col].isnull(), col] = finalTrain[col].median()

In [9]:
fill_nulls(0)

In [10]:
def fill_nulls(value):
    cols_fill = ['mths_since_last_record','mths_since_last_major_derog',
                 'mths_since_last_delinq','total_rev_hi_lim','tot_cur_bal',
                 'tot_coll_amt','emp_length','revol_util','collections_12_mths_ex_med',
                 'open_acc','total_acc','acc_now_delinq','avl_lines','loan_to_income',
                 'annual_inc','bad_state','total_repayment_progress']
    if value == -9999:
        for col in cols_fill:
            finalTrain.loc[finalTrain[col].isnull(), col] = -9999
    else : 
        for col in cols_fill:
            finalTrain.loc[finalTrain[col].isnull(), col] = finalTrain[col].median()

In [11]:
fill_nulls(0)

In [12]:
finalTrain = finalTrain.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_trainScaled = scaler.fit_transform(finalTrain)


In [14]:

# Split train and cross validation sets
X_train, X_test, y_train, y_test = train_test_split(np.array(finalTrain), np.array(train_target), test_size=0.30)
eval_set=[(X_test, y_test)]

In [15]:
print('Initializing xgboost.sklearn.XGBClassifier....')


clf = xgboost.sklearn.XGBClassifier(
    objective="binary:logistic", 
    learning_rate=0.09, 
    seed=9616, 
    max_depth=30, 
    gamma=10, 
    n_estimators=500)



Initializing xgboost.sklearn.XGBClassifier....


In [16]:
print("Training Started")
clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=eval_set, verbose=True)

y_pred = clf.predict(X_test)
accuracy = accuracy_score(np.array(y_test).flatten(), y_pred)




Training Started


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[0]	validation_0-auc:0.96856
[1]	validation_0-auc:0.97034
[2]	validation_0-auc:0.97111
[3]	validation_0-auc:0.97209
[4]	validation_0-auc:0.97253
[5]	validation_0-auc:0.97265
[6]	validation_0-auc:0.97282
[7]	validation_0-auc:0.97299
[8]	validation_0-auc:0.97304
[9]	validation_0-auc:0.97324
[10]	validation_0-auc:0.97334
[11]	validation_0-auc:0.97346
[12]	validation_0-auc:0.97358
[13]	validation_0-auc:0.97373
[14]	validation_0-auc:0.97375
[15]	validation_0-auc:0.97383
[16]	validation_0-auc:0.97394
[17]	validation_0-auc:0.97417
[18]	validation_0-auc:0.97423
[19]	validation_0-auc:0.97428
[20]	validation_0-auc:0.97437
[21]	validation_0-auc:0.97443
[22]	validation_0-auc:0.97445
[23]	validation_0-auc:0.97449
[24]	validation_0-auc:0.97453
[25]	validation_0-auc:0.97454
[26]	validation_0-auc:0.97460
[27]	validation_0-auc:0.97465
[28]	validation_0-auc:0.97472
[29]	validation_0-auc:0.97475
[30]	validation_0-auc:0.97478
[31]	validation_0-auc:0.97484
[32]	validation_0-auc:0.97490
[33]	validation_0-au

In [17]:
submission_file_name = "Loan_Defaulter_submission"
print("Accuracy: %.10f%%" % (accuracy * 100.0))
submission_file_name = submission_file_name + ("_Accuracy_%.6f" % (accuracy * 100)) + '_'

accuracy_per_roc_auc = roc_auc_score(np.array(y_test).flatten(), y_pred)
print("ROC-AUC: %.10f%%" % (accuracy_per_roc_auc * 100))
submission_file_name = submission_file_name + ("_ROC-AUC_%.6f" % (accuracy_per_roc_auc * 100))



Accuracy: 94.1344402081%
ROC-AUC: 92.5528901415%


### Final Submission

In [18]:
final_pred = pd.DataFrame(clf.predict_proba(np.array(finalTest)))
dfSub = pd.concat([test_member_id, final_pred.iloc[:, 1:2]], axis=1)
dfSub.rename(columns={1:'loan_status'}, inplace=True)
dfSub.to_csv((('%s.csv') % (submission_file_name)), index=False)

